In [3]:
# https://plot.ly/python/subplots/
import plotly as py
import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs

py.offline.init_notebook_mode(connected=True)

import numpy as np
from sklearn.metrics import mean_squared_error

def print_mse(got, predicted, prefix=''):
    mse = mean_squared_error(got, predicted)
    if isinstance(mse, np.complex):
        print('%s Mean Squared Error: (%.2g, %.2g)' % (prefix, mse.real, mse.imag))
    else:
        print('%s Mean Squared Error: %.2g' % (prefix, mse))
        



In [46]:
# фильтр скользящего среднего
def create_filter_median(kernel_size):
    if kernel_size % 2 == 0:
        raise ValueError("kernel_size should be odd")
    return [1 / kernel_size for i in range(kernel_size)]
    
def apply_filter_median(signal, filt):
    return np.convolve(signal, filt, mode='valid')

def impulse_response_median(filt):
    f = np.pad(filt, (0, len(filt) * 4), 'constant', constant_values=(0,0))
    return np.fft.fft(f)[:int(len(f) / 2)]


In [3]:
N = 500
kernel_size = 71
T = 1.0 # s

t = np.linspace(0, T, N)
s = np.random.rand(N)

filt = create_filter_median(kernel_size)
fs = apply_filter_median(s, filt)
ir = impulse_response_median(filt)

tr_s = go.Scatter(
    x = t,
    y = s,
    name='signal'
)
tr_fs = go.Scatter(
    x = t,
    y = fs,
    name='filtered'
)

tr_ir = go.Scatter(
    x = [i for i in range(len(ir.real))],
    y = np.absolute(ir),
    name='impulse response'
)


fig = py.tools.make_subplots(rows = 2, cols = 1)

fig['layout'].update(height=900, width=900)

fig.append_trace(tr_s, 1, 1)
fig.append_trace(tr_fs, 1, 1)

fig.append_trace(tr_ir, 2, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [10]:
def create_filter_butterworth(signalLength, sampleFreq, order, f0, Gain):
    nBins = signalLength // 2
    binWidth = sampleFreq / signalLength
    
    filt = np.array([0 for i in range(nBins)], dtype=complex)
    
    for i in range(nBins):
        binFreq = binWidth * i
        gain = Gain / np.sqrt(1 + np.power(binFreq / f0, 2 * order))
        filt[i] = gain
        
#     df = 2 * np.pi / f0
#     filt[i] *= np.exp(1j * df * i)
    
    rect = [1 if i < 10 else 0 for i in range(signalLength)]
    rect = np.fft.fft(rect)[:len(rect) // 2]
    
    for i in range(len(filt)):
        filt[i] *= np.exp(1j * -np.angle(rect[i]))
    return filt

def apply_filter_butterworth(s, filt):
    N = len(s)
    ffts = np.fft.fft(s)
    
    nBins = N // 2
    
    for i in range(nBins):
        ffts[i] *= filt[i]
        ffts[N - i - 1] *= filt[i]
    return np.real(np.fft.ifft(ffts))

def impulse_response_butterworth(filt):
    return filt

def prod(iterable):
    from functools import reduce
    from operator import mul
    return reduce(mul, iterable, 1)

def transfer_function_butterworth(Rp, Rs, w0, w1):
    ep = np.sqrt(np.power(10, Rp / 10.) - 1)
    es = np.sqrt(np.power(10, Rs / 10.) - 1)
    k = w0 / w1
    k1 = ep / es
    N = int(np.ceil(np.log(k1) / np.log(k)))
    L = N // 2
    r = N % 2
    a = np.power(ep, - 1. / N)
    th = [(2 * n + 1) / 2 / N * np.pi for n in range(L)]
    
    return lambda s: 1. / (ep * np.power(s + a, r) * prod(s*s + 2*a*np.sin(thn)*s + a*a for thn in th))


# def butterworth_filter(s, sampleFreq, order, f0, Gain):
#     N = len(s)
#     ffts = np.fft.fft(s)
#     
#     nBins = N // 2
#     binWidth = sampleFreq / N
#     
#     for i in range(1, nBins):
#         binFreq = binWidth * i
#         gain = Gain / np.sqrt(1 + np.power(binFreq / f0, 2 * order))
#         ffts[i] *= gain
#         ffts[N - i] *= gain
#     return np.real(np.fft.ifft(ffts))


In [110]:
N = 20
T = 1.0 * N # s

Fs = 25
f1 = 1
f2 = 8
w1 = 2 * np.pi * f1
w2 = 2 * np.pi * f2

t = np.linspace(0, T, N)
s = [np.sin(w1 / Fs * i) + np.sin(w2 / Fs * i) for i in t]

filt = create_filter_butterworth(N, 25, 3, 1.5, 1.0)
fs = apply_filter_butterworth(s, filt)
# fs = butterworth_filter(s, 25, 3, 1.5, 1.0)
ir = impulse_response_butterworth(filt)

sym = np.append(ir[::-1], ir)

coeffs = np.fft.ifft(sym)
coeffs = coeffs[:len(coeffs) // 2]

tr_s = go.Scatter(
    x = t,
    y = s,
    name='signal'
)
tr_fs = go.Scatter(
    x = t,
    y = fs,
    name='filtered'
)

tr_ir = go.Scatter(
    x = [i for i in range(len(ir.real))],
    y = np.absolute(ir),
    name='impulse response'
)

tr_ir_sym = go.Scatter(
    x = [i for i in range(len(sym.real))],
    y = np.absolute(sym),
    name='sym'
)

tr_ir_phase = go.Scatter(
    x = [i for i in range(len(ir.real))],
    y = np.angle(ir) * 0.2,
    name='angle'
)

tr_coeffs = go.Scatter(
    x = t,
    y = np.absolute(coeffs),
    name = 'coeffs'
)


fig = py.tools.make_subplots(rows = 3, cols = 1)

fig['layout'].update(height=900, width=900)

fig.append_trace(tr_ir, 1, 1)
fig.append_trace(tr_ir_phase, 1, 1)
# fig.append_trace(tr_ir_sym, 1, 1)
fig.append_trace(tr_coeffs, 2, 1)

fig.append_trace(tr_s, 3, 1)
fig.append_trace(tr_fs, 3, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]



In [30]:
N = 50
T = 10.0 # s

t = np.linspace(0, T, N)
s = np.ones(N)

df = 1. / T
k = [i * df for i in range(N)]

HGen = transfer_function_butterworth(1, 30, 1, 5)

tr_afr = go.Scatter(
    x = k,
    y = [np.absolute(HGen(1j*i)) for i in k],
    name='signal'
)

tr_phase = go.Scatter(
    x = k,
    y = [np.angle(HGen(1j*i)) for i in k],
    name='signal'
)

rows = 2
cols = 1
fig = py.tools.make_subplots(rows = rows, cols = cols)

fig['layout'].update(height=rows * 450, width=cols * 900)

fig.append_trace(tr_afr, 1, 1)
fig.append_trace(tr_afr, 2, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]

